In [1]:
import torch 
from torch import nn

# class DetectorPanel(nn.Module):
#     def __init__(self, xy, z, zfromhodoscope):
#         super().__init__()
#         self.xy = xy
#         self.z = z
#         self.zfromhodoscope = zfromhodoscope

#     def globalz(self):
#         return self.z+self.zfromhodoscope
    
#     def print(self):
#         #print(f"\t Panel has xy={self.xy.data}, z={self.z.data+self.globalz.data}")
#         print(f"\t Panel has xy={self.xy.data}, relative z={self.z.data}, z from hodoscope={self.zfromhodoscope.data}, global z={self.globalz()}")
 
# class Hodoscope(nn.Module):
#     def __init__(self, xy, z):
#         super().__init__()
#         self.xy = nn.parameter.Parameter(data=xy, requires_grad=True)
#         self.z = nn.parameter.Parameter(data=z, requires_grad=True)
#         self.hodoscopes = []
        
#     def generatehodoscopes(self, relzs):
#         self.positionList= [nn.parameter.Parameter(data=i, requires_grad=False) for i in relzs]
#         for z in self.positionList:
#             self.hodoscopes.append(DetectorPanel(self.xy,z, self.z))

#     def print(self):
#         print(f"Hodoscope has xy={self.xy.data}, z={self.z.data}")
            
#     def printhodoscopes(self):
#         for panel in self.panels:
#             panel.print()
            
            
#     def modifyPosition(self, xy, z):
#         #self.xy = nn.parameter.Parameter(xy)
#         #self.z = nn.parameter.Parameter(z)
#         self.xy.data = xy
#         self.z.data = z

    

# # hod = Hodoscope(torch.Tensor([2,3]), torch.Tensor([5]))
# # positionList=torch.Tensor([0.,0.3,0.9])

# # hod.generatePanels(positionList)
# # hod.print()

# # hod.printPanels()

# # hod.modifyPosition(torch.Tensor([1,2]), torch.Tensor([8]))

# # hod.print()
# # hod.printPanels()

## Layers and hodoscopes


The default *layer* class `PanelDetectorLayer` in TomOpt uses multiple *panels* `DetectorPanel` to record muon position.

`PanelDetectorLayer` inherits from the `AbsDetectorLayer` and **MUST** provide the following **methods**:

 - `forward`: The forward method to propagate the muons. It is a central aspect of `nn.Module`.
 - `get_cost`
 - `conform_detector`
  - `assign_budget`

and the following **features**:

 - `pos`: the position of the layer, either `above` or `bellow`. 
 - `lw`: the length and width of the layer.
 - `z`: the z position of the top of gthe layer. 
 - `size`: the heigth of the layer such that z - size is the bottom of the layer.
 - `device`: torch.device.

In [30]:
from tomopt.volume.layer import AbsDetectorLayer
from tomopt.volume.panel import DetectorPanel
from typing import List, Union, Optional, Iterator, Tuple
from torch import Tensor
import numpy as np
from tomopt.muon import MuonBatch

from tomopt.core import DEVICE

class Hodoscope(nn.Module):

    def __init__(self,
                 *,
                 init_xyz: Tuple[float, float, float],
                 init_xyz_span: Tuple[float, float, float],
                 xyz_gap: Tuple[float, float, float],
                 n_panels: int = 3,
                 res: float = 1000,
                 eff: float = 0.9,
                 m2_cost: float = 1.,
                 budget: Optional[Tensor] = None,
                 realistic_validation: bool = True,
                 device: torch.device = DEVICE):
        
        super().__init__()
        self.realistic_validation, self.device = realistic_validation, device
        # self.register_buffer("m2_cost", torch.tensor(float(m2_cost), device=self.device))
        self.xy = nn.Parameter(torch.tensor(init_xyz[:2], device=self.device))
        self.z = nn.Parameter(torch.tensor(init_xyz[-1], device=self.device))
        self.xyz_span = nn.Parameter(torch.tensor(init_xyz_span, device=self.device))
        self.xyz_gap = xyz_gap
        self.n_panels = n_panels
        self.res = res
        self.eff = eff
        self.panels = self.generate_init_panels()

        self.device = device
    
    def __getitem__(self, idx: int) -> DetectorPanel:
        return self.panels[idx]

    def generate_init_panels(self) -> List[DetectorPanel]:

        r"""
        Generates Detector panels based on the xy and z position (xy, z), the span of the hodoscope (xyz_span), 
        and the gap between the edge of the hodoscope and the panels (xyz_gap).

        Returns:
            DetectorPanles as a nn.ModuleList.
        """
        
        return nn.ModuleList(
            [DetectorPanel(res=self.res, 
                           eff=self.eff,
                           init_xyz=[self.xy[0],
                                     self.xy[1],
                                     self.z - self.xyz_gap[2] - (self.xyz_span[2]-2*self.xyz_gap[2])*i/(self.n_panels-1)], 
                           init_xy_span=[self.xyz_span[0] - 2 * self.xyz_gap[0], self.xyz_span[1] - 2 * self.xyz_gap[1]],
                           device=DEVICE) for i in range(self.n_panels)])

    def get_cost(self) -> Tensor:

        return torch.sum([p.get_cost() for p in self.panels])

class HodoscopeDetectorLayer(AbsDetectorLayer):

    def __init__(self, 
                 pos:str, 
                 *,
                 lw:Tensor,
                 z:float,
                 size:float, 
                 hodoscopes: List[Hodoscope],
    ):
        if isinstance(hodoscopes, list):
            hodoscopes = nn.ModuleList(hodoscopes)

        super().__init__(pos=pos, lw=lw, z=z, size=size, device=self.get_device(hodoscopes))
        self.hodoscopes = hodoscopes

        if isinstance(hodoscopes[0], Hodoscope):
            self.type_label = "hodoscope"
            self._n_costs = len(self.hodoscopes)

    @staticmethod
    def get_device(hodoscopes: nn.ModuleList) -> torch.device:

        r"""
        Helper method to ensure that all panels are on the same device, and return that device.
        If not all the panels are on the same device, then an exception will be raised.

        Arguments:
            panels: ModuleLists of either :class:`~tomopt.volume.panel.DetectorPanel` or :class:`~tomopt.volume.heatmap.DetectorHeatMap` objects on device

        Returns:
            Device on which all the panels are.
        """

        device = hodoscopes[0].device
        if len(hodoscopes) > 1:
            for h in hodoscopes[1:]:
                if h.device != device:
                    raise ValueError("All hodoscopes must use the same device, but found multiple devices")
        return device
    
    def get_panel_zorder(self) -> List[int]:
        r"""
        Returns:
            The indices of the panels in order of decreasing z-position.
        """

        return list(np.argsort([p.z.detach().cpu().item() for h in self.hodoscopes for p in h.panels])[::-1])
    
    def yield_zordered_panels(self) -> Iterator[Tuple[int, DetectorPanel]]:
        r"""
        Yields the index of the panel, and the panel, in order of decreasing z-position.

        Returns:
            Iterator yielding panel indices and panels in order of decreasing z-position.
        """
        panels = [p for h in self.hodoscopes for p in h.panels]

        for i in self.get_panel_zorder():
             yield i, panels[i]

    def forward(self, mu: MuonBatch) -> None:
        r"""
        Propagates muons to each detector panel, in order of decreasing z-position, and calls their `get_hits` method to record hits to the muon batch.
        After this, the muons will be propagated to the bottom of the detector layer.

        Arguments:
            mu: the incoming batch of muons
        """

        for i, p in self.yield_zordered_panels():
            mu.propagate_dz(mu.z - p.z.detach())  # Move to panel
            hits = p.get_hits(mu)
            mu.append_hits(hits, self.pos)
        mu.propagate_dz(mu.z - (self.z - self.size))  # Move to bottom of layer

    def get_cost(self) -> Tensor:
        r"""
        Returns the total, current cost of the detector(s) in the layer, as computed by looping over the hodoscopes and summing the returned values of calls to
        their `get_cost` methods.

        Returns:
            Single-element tensor with the current total cost of the detector in the layer.
        """

        cost = None
        panels = [p for h in self.hodoscopes for p in h.panels]
        for p in panels:
            cost = p.get_cost() if cost is None else cost + p.get_cost()
        return cost

In [36]:
hod1 = Hodoscope(init_xyz= [.5, .5, 1.], 
                 init_xyz_span = [1., 1., .3], 
                 xyz_gap = [.1, .1, .1])

hod2 = Hodoscope(init_xyz = [0.25, 0.25, 0.6], 
                 init_xyz_span = [1., 1., .3], 
                 xyz_gap = [.1, .1, .1])


hods = [hod1, hod2]

hod_detector_above = HodoscopeDetectorLayer(pos = 'above', lw = Tensor([1,1]), z = 1., size = 0.5, 
                                            hodoscopes = hods)
